In [1]:
import pandas as pd
from sqlalchemy import create_engine

#database_credentials
user = 'root'
host = 'localhost'
password = '2004'
database = 'airport_database'

#create connection engine
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')

#query data
query = "select * from airportdata limit 10000"


df = pd.read_sql(query, engine)
df.head()

,id,Origin_airport,Destination_airport,Origin_city,Destination_city,Passengers,Seats,Flights,Distance,Fly_date,Origin_population,Destination_population,Org_airport_lat,Org_airport_long,Dest_airport_lat,Dest_airport_long
0,1,EUG,RDM,"Eugene, OR","Bend, OR",41,396,22,103,1990-11-01,284093,76034,44.124599,-123.211998,44.254101,-121.150002
1,2,EUG,RDM,"Eugene, OR","Bend, OR",88,342,19,103,1990-12-01,284093,76034,44.124599,-123.211998,44.254101,-121.150002
2,3,EUG,RDM,"Eugene, OR","Bend, OR",11,72,4,103,1990-10-01,284093,76034,44.124599,-123.211998,44.254101,-121.150002
3,4,MFR,RDM,"Medford, OR","Bend, OR",0,18,1,156,1990-02-01,147300,76034,42.374199,-122.873001,44.254101,-121.150002
4,5,MFR,RDM,"Medford, OR","Bend, OR",11,18,1,156,1990-03-01,147300,76034,42.374199,-122.873001,44.254101,-121.150002


In [2]:
# Import required libraries
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# Load your data (replace with your actual data loading)
# df = pd.read_csv('your_data.csv')
# Sample data structure should include:
# 'Origin_airport', 'Destination_airport', 'Fly_date', 'Distance', 'Passengers', 'Seats'

# Create Dash app
app = Dash(__name__)

# Initial figures (for first load)
initial_fig = px.scatter(
    df, 
    x='Distance', 
    y='Passengers', 
    color='Origin_airport',
    title='Passengers vs Distance',
    labels={'Passengers': 'Passengers', 'Distance': 'Distance (Miles)'}
)

initial_fig_bar = px.bar(
    df,
    x="Origin_airport",
    y="Seats",
    color="Destination_airport",
    title="Seats per Flight",
    labels={"Seats": "No of Seats", "Origin_airport": "Origin Airport"}
)

# App layout
app.layout = html.Div(children=[
    html.H1(children='Flight Dashboard'),
    html.Div(children='Dashboard for visualization of flight data'),
    
    # Controls
    html.Div([
        dcc.Dropdown(
            id='origin-airport-dropdown',
            options=[{'label': airport, 'value': airport} 
                    for airport in df['Origin_airport'].unique()],
            placeholder="Select Origin Airport(s)",
            multi=True
        ),
        dcc.DatePickerRange(
            id='date-picker-range',
            start_date=df['Fly_date'].min(),
            end_date=df['Fly_date'].max(),
            display_format='YYYY-MM-DD'
        )
    ], style={'padding': '20px'}),
    
    # Graphs
    dcc.Graph(
        id='passenger-distance-graph',
        figure=initial_fig
    ),
    dcc.Graph(
        id='seats-bar-chart',
        figure=initial_fig_bar
    )
])

# Callback for interactive filtering
@app.callback(
    [Output('passenger-distance-graph', 'figure'),
     Output('seats-bar-chart', 'figure')],
    [Input('origin-airport-dropdown', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)
def update_graphs(selected_airports, start_date, end_date):
    # Filter by date range first
    filtered_df = df[
        (df['Fly_date'] >= start_date) & 
        (df['Fly_date'] <= end_date)
    ]
    
    # Filter by selected airports if any are selected
    if selected_airports:
        filtered_df = filtered_df[filtered_df['Origin_airport'].isin(selected_airports)]
    
    # Create updated figures
    scatter_fig = px.scatter(
        filtered_df, 
        x='Distance', 
        y='Passengers', 
        color='Origin_airport',
        title='Filtered Passengers vs Distance',
        labels={'Passengers': 'Passengers', 'Distance': 'Distance (Miles)'}
    )
    
    bar_fig = px.bar(
        filtered_df, 
        x='Origin_airport', 
        y='Seats', 
        color='Destination_airport',
        title='Filtered Seats per Flight',
        labels={"Seats": "No of Seats", "Origin_airport": "Origin Airport"}
    )
    
    return scatter_fig, bar_fig

# Run the app
if __name__ == '__main__':
    app.run(debug=True)
    